In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [9]:
import pandas as pd
df = pd.read_csv("combined_data.csv")
df.head(5)

,text,label
0,#Depressed mood can be caused by infectious di...,0
1,"With all of this unnessary family drama, I fe...",1
2,"I'm going to keep banging on about this, cos i...",0
3,Ugh. Woke up to the same acne covered face thi...,0
4,#stoner #instahookah #depressed Portable Trans...,0


In [10]:
df.groupby('label').describe()

text                                                               
       count unique                                                top freq
label                                                                      
0      11470   9985    haIZ....frOM laSt wEek i keEP on goiNg out.....    6
1       5709   4373  Mom's depression tied to kids' emotional, inte...   11

In [11]:
df_positive=df[df['label']==1]
df_negative=df[df['label']==0]
df_negative_down = df_negative.sample(df_positive.shape[0])
df_negative_down.shape

df_balanced = pd.concat([df_positive,df_negative_down])
df_balanced

,text,label
1,"With all of this unnessary family drama, I fe...",1
6,"I feel so sad because i can't be happy, and th...",1
7,I went to church with my mom and a huge chunk ...,1
8,No love here.... #lonely #depressed pic.twitt...,1
9,I see so many people moving forward with their...,1
...,...,...
2162,my head is like a 2 year old's finger painti...,0
9204,@_BellaCullen18_ hey Bella,0
4951,not invited to friends party yet others who ...,0
13356,@kyujaemin coughcough. still on season one. o....,0


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['text'],df_balanced['label'],stratify=df_balanced['label'])

In [13]:
X_train.head(10)

15996    Exercise and Depression: How Sweating Is Good ...
16000    The new Sonic Mania trailer cured my depressio...
194      They say life's short, but I'm bored already. ...
5637      i’m so tired i don’t know what i’m doing in l...
17174    Many sufferers of depression aren't sad; they ...
2985      this is so stupid  i have gone over and over ...
14545    Holding squats for a minute each... OMG. My fr...
16220    Having a depression is so hard . You need to f...
15151    When Friends Struggle With #Depression, This A...
4666      GAL)entines day is 48 hours today  take this ...
Name: text, dtype: object

In [14]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [15]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embedding(["$110 discount, hurry up","Bhavin, are you up for a football game tonight?"])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8321506 , -0.49884495, -0.83281654, ..., -0.63157123,
        -0.7261329 ,  0.9110628 ],
       [-0.85999304, -0.5069298 , -0.94539154, ..., -0.870103  ,
        -0.6732965 ,  0.82944727]], dtype=float32)>

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
#bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
embeddings = bert_encoder(preprocessed_text)
print(embeddings['pooled_output'])
#neural network layers
l = tf.keras.layers.Dropout(0.1, name='dropout')(embeddings['pooled_output'])

l = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

#construct final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='keras_layer_1/StatefulPartitionedCall:13', description="created by layer 'keras_layer_1'")


In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [19]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

In [ ]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
264/268 [============================>.] - ETA: 38s - loss: 0.5800 - accuracy: 0.7016 - precision: 0.7004 - recall: 0.7058

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np
+